In [1]:
import scrapy
import logging
from scrapy.crawler import CrawlerProcess

In [2]:
class ProductsSpider(scrapy.Spider):
    
    name = 'Products'
    start_urls = ['https://www.perfumeria.pl/collections/perfumy'] 
    
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'FEEDS': {'res2.csv': {'format':'csv'}}
        ,'DOWNLOAD_DELAY': 3 
        ,'RANDOMIZE_DOWNLOAD_DELAY' : True 
    }
    
    def parse_product(self,response):
        
        xpath_name = '//h1[@class="product__title"]/text()'  
        xpath_EAN = "//script[@type='text/javascript' and text()[contains(.,'barcode')]]/text()" 
        xpath_price_promo =  '//span[@class = "price-item price-item--sale price-item--last"]/text()'  
        xpath_price =  '//s[@class = "price-item price-item--regular"]/text()' 
        
        name = response.xpath(xpath_name).get()
        EAN = response.xpath(xpath_EAN).getall()[-1].split("barcode")[1].split("}]")[0].replace('":',"").replace('"',"")
        price_promo = response.xpath(xpath_price_promo).get().replace(" zł PLN","").replace(",",".")
        price = response.xpath(xpath_price).get().replace(" zł PLN","").replace(",",".").strip()
        
        yield {
                'Name': name,
                'EAN': EAN,
                'price': price,
                'price promo': price_promo,
                'url': response.url
                }

    def parse(self,response):
        
        xpath_url = '//a[@class = "full-unstyled-link"]/@href' # link to the product
        xpath_next_page = '//link[@rel = "next"]/@href' # link to next page
        
        selection = response.xpath(xpath_url)
        
        for s in selection: 
            href = 'https://www.perfumeria.pl' + s.get()
            yield scrapy.Request(href, self.parse_product) 
            
        next_page = response.xpath(xpath_next_page).get()
        if next_page:
            yield response.follow('https://www.perfumeria.pl' + next_page, callback=self.parse)

In [1]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.34 Safari/537.36'
})

process.crawl(ProductsSpider)
process.start()